# RSS Infomation Research

In [1]:
# RSS에서 xml태그 별 정보 추출
!pip install feedparser 

# 인터넷 신문기사 분석(라이브러리 내의 "Article()"사용하기 위해)
!pip install newspaper3k 

# 한국어 형태소 분석기 (주어진 문장에서 명사만 추출할 목적으로 사용)
!pip install konlpy 

# 라이브러리 불러오기

In [4]:
import feedparser # xml 태그별 정보 추출
from newspaper import Article # 인터넷 신문 기사 분석
from konlpy.tag import Okt # 한국어 형태소 분석기
from collections import Counter # 갯수 세기(TF계산용)
from operator import eq # 동일한 단어인지 여부 확인 위해 사용
from bs4 import BeautifulSoup # 글에 존재할지 모르는 html 태그 삭제를 위해 사용

# 정보 수집할 rss 링크 설정 및 정보 수집하기

##### 전자신문 (https://www.etnews.com/rss/) 의 오늘의 뉴스 부분 

##### 함수 정의

In [5]:
# 제목, 링크 수집 함수
def crawl_rss(urls):
    array_rss = [] # 모든 기사를 넣을 리스트
    titles_rss = set() # 중복된 기사를 제거하기 위한 기사 제목들 집합
    
    for url in urls: # url 순회
        print(f"[Crawl Rss] {url}") # 현재 크롤링하는 rss 링크 출력
        parser_rss = feedparser.parse(url) # 현재 url을 파싱해서, 그 결과를 저장
        for p in parser_rss.entries:
            if p.title not in titles_rss: # 중복되지 않은 기사 일 경우 if 문 실행
                array_rss.append({'title':p.title, 'link':p.link})
                titles_rss.add(p.title)
#             else:
#                 print(f"Duplicated Title : {p.title}")
    return array_rss

# 링크를 통해 제목, 본문 내용 추출
def crawl_article(url, language='ko'):
    print(f"[Crawling. . .] {url}") # 현재 크롤링하는 링크 출력
    news_paper = Article(url, language=language)
    news_paper.download()
    news_paper.parse()
    return news_paper.title, news_paper.text

In [6]:
# 정보 수집할 rss 링크 모음
urls = ['http://rss.etnews.com/Section901.xml',
        'http://rss.etnews.com/Section902.xml',
        'http://rss.etnews.com/Section903.xml',
        'http://rss.etnews.com/Section904.xml']

# 제목과 기사 링크 수집하기
list_articles = crawl_rss(urls)

# 수집한 (제목,기사링크)를 토대로 제목과 본문 내용 수집하기
for article in list_articles:
    _, text = crawl_article(article['link'])
    article['text'] = text

[Crawl Rss] http://rss.etnews.com/Section901.xml
[Crawl Rss] http://rss.etnews.com/Section902.xml
[Crawl Rss] http://rss.etnews.com/Section903.xml
[Crawl Rss] http://rss.etnews.com/Section904.xml
[Crawling. . .] https://www.etnews.com/20230926000183
[Crawling. . .] https://www.etnews.com/20230926000128
[Crawling. . .] https://www.etnews.com/20230926000101
[Crawling. . .] https://www.etnews.com/20230926000070
[Crawling. . .] https://www.etnews.com/20230926000033
[Crawling. . .] https://www.etnews.com/20230926000022
[Crawling. . .] https://www.etnews.com/20230924000164
[Crawling. . .] https://www.etnews.com/20230926000288
[Crawling. . .] https://www.etnews.com/20230926000275
[Crawling. . .] https://www.etnews.com/20230926000231
[Crawling. . .] https://www.etnews.com/20230926000185
[Crawling. . .] https://www.etnews.com/20230926000069
[Crawling. . .] https://www.etnews.com/20230926000282
[Crawling. . .] https://www.etnews.com/20230926000274
[Crawling. . .] https://www.etnews.com/202309260

# 수집 내용 확인하기

In [7]:
list_articles[0]

{'title': '[사설]공공 정보시스템 안정성 제고하길',
 'link': 'https://www.etnews.com/20230926000183',
 'text': "행정·공공기관이 보유한 정보시스템 운영시설의 건물, 전기, 소방·방재 등 안정성 기준이 강화된다.\n\n\n\n행정안전부가 행정·공공기관이 각종 재난과 재해에도 디지털플랫폼정부(디플정) 서비스를 안정적으로 제공하도록 정보시스템 운영시설 안정성 기준을 반영한 '행정기관 및 공공기관 정보자원 통합기준'을 개정해 12월부터 시행한다.\n\n\n\n새로운 안정성 기준은 건물, 전기, 소방·방재 등 7개 분야 67개 항목이다. 정보시스템 등급별로 운영시설 안정성 수준을 상·중·하로 나누고 점검 항목을 차등 적용했다.\n\n\n\n점검 항목은 지진에 대비한 내진 적용 여부, 화재를 대비한 소방 화재 진압, 평상시 운영·유지관리 매뉴얼, 업무연속성계획(BCP) 등을 포함했다. 정보시스템 등급에 따라 정보시스템 등급 1·2 등급은 운영시설 안정성 '상' 기준을, 3등급은 '중' 기준을, 4·5 등급은 '하' 이상 기준을 충족해야 한다.\n\n\n\n행안부는 행정·공공기관 자체 점검결과를 바탕으로 필요하면 현장점검을 실시할 예정이다. 안정성 점검기준을 충족하지 않는 경우에는 개선도 권고한다.\n\n\n\n행정·공공기관이 차제에 경각심을 상기하고, 성실하게 실천해야 한다. 행정·공공기관의 정보시스템 운영시설에 문제가 발생하면 디플정 서비스는 물론 국민 일상 생활에도 불편을 초래할 수 밖에 없다. 자연재해이든 인재이든 국가적 손실을 감수해야 하는 만큼 안정성은 아무리 강조해도 지나치지 않는다.\n\n\n\n행안부가 행정·공공기관의 이행을 독려하고, 미진한 부분을 지속 보완하도록 철저하게 파악하고, 꼼꼼하게 점검하길 바란다. 행정·공공기관의 안전의식이 높아졌다고 하지만 만족해선 안된다."}

# 모든 본문 text에서 명사  (키워드, 빈도수) 추출

##### 키워드와 빈도수 추출하는 함수 정의

In [8]:
def get_keywords_and_frequency(text,nKeywords=10): #텍스트와  추출할 키워드 개수
    spliter = Okt() # Okt 객체 생성
    nouns = spliter.nouns(text) # 명사 추출 
    count = Counter(nouns) # 명사별 빈도수 추출
    list_keywords = []
    
    for n,c in count.most_common(nKeywords): # 최빈값 높은 명사 순으로, nKeywords까지 출력
        item = {'keyword':n, 'count':c}
        list_keywords.append(item)
    return list_keywords

##### 추출 내용 저장하기

In [10]:
for article in list_articles:
    keywords = get_keywords_and_frequency(article['text'])
    article['keywords'] = keywords

# 검색어를 입력받아서 그 검색어를 가지고 있는 기사를 출력하기

In [19]:
# 검색어와 동일한 키워드가 있을 때, 빈도수 반환
def search_articles(query, list_keywords): # 쿼리가 키워드 리스트에 있으면 출력 (없으면 0)
    nWords = 0 # 키워드 빈도수 초기화
    for kw in list_keywords:
        if eq(query, kw['keyword']):
            nWords = kw['count']
    return nWords

In [22]:
query = input("키워드를 입력하세요 : ")

for article in list_articles:
    nQuery = search_articles(query, article['keywords'])
    if nQuery != 0:
        print(f"[TF] {nQuery}, [Title] {article['title']}, [URL] {article['link']}")

키워드를 입력하세요 : 반도체
[TF] 20, [Title] 반도체·디스플레이의 '새 시대(New Era)'…전자신문 테크서밋 내달 18일 개막, [URL] https://www.etnews.com/20230926000282
[TF] 10, [Title] [에듀플러스]제2회 도쿄일렉트론코리아 드림업 반도체 캠프, 10월 21일·11월 25일 개최…무료 참가 신청은 이티에듀 홈페이지, [URL] https://www.etnews.com/20230925000316
[TF] 25, [Title] "中서 현상 유지만 하라는 것…첨단 반도체 대규모 투자 어려워", [URL] https://www.etnews.com/20230924000040
[TF] 8, [Title] 美, '반도체법' 가드레일 규정 최종 발표, [URL] https://www.etnews.com/20230923000006
[TF] 6, [Title] 펨트론, 웨이퍼 표면 검사 장비 국산화, [URL] https://www.etnews.com/20230925000191
[TF] 10, [Title] “美 반도체 보조금 받으면 중국 내 생산능력 5%만 확장 가능”, [URL] https://www.etnews.com/20230922000447


In [23]:
list_articles[0]

{'title': '[사설]공공 정보시스템 안정성 제고하길',
 'link': 'https://www.etnews.com/20230926000183',
 'text': "행정·공공기관이 보유한 정보시스템 운영시설의 건물, 전기, 소방·방재 등 안정성 기준이 강화된다.\n\n\n\n행정안전부가 행정·공공기관이 각종 재난과 재해에도 디지털플랫폼정부(디플정) 서비스를 안정적으로 제공하도록 정보시스템 운영시설 안정성 기준을 반영한 '행정기관 및 공공기관 정보자원 통합기준'을 개정해 12월부터 시행한다.\n\n\n\n새로운 안정성 기준은 건물, 전기, 소방·방재 등 7개 분야 67개 항목이다. 정보시스템 등급별로 운영시설 안정성 수준을 상·중·하로 나누고 점검 항목을 차등 적용했다.\n\n\n\n점검 항목은 지진에 대비한 내진 적용 여부, 화재를 대비한 소방 화재 진압, 평상시 운영·유지관리 매뉴얼, 업무연속성계획(BCP) 등을 포함했다. 정보시스템 등급에 따라 정보시스템 등급 1·2 등급은 운영시설 안정성 '상' 기준을, 3등급은 '중' 기준을, 4·5 등급은 '하' 이상 기준을 충족해야 한다.\n\n\n\n행안부는 행정·공공기관 자체 점검결과를 바탕으로 필요하면 현장점검을 실시할 예정이다. 안정성 점검기준을 충족하지 않는 경우에는 개선도 권고한다.\n\n\n\n행정·공공기관이 차제에 경각심을 상기하고, 성실하게 실천해야 한다. 행정·공공기관의 정보시스템 운영시설에 문제가 발생하면 디플정 서비스는 물론 국민 일상 생활에도 불편을 초래할 수 밖에 없다. 자연재해이든 인재이든 국가적 손실을 감수해야 하는 만큼 안정성은 아무리 강조해도 지나치지 않는다.\n\n\n\n행안부가 행정·공공기관의 이행을 독려하고, 미진한 부분을 지속 보완하도록 철저하게 파악하고, 꼼꼼하게 점검하길 바란다. 행정·공공기관의 안전의식이 높아졌다고 하지만 만족해선 안된다.",
 'keywords': [{'keyword': '공공기관', 'count': 8},
  {'keyword': '안정', 'co